# **NLP2_Run3**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [ ]:
problem = """ You are a chemist in charge of a process that requires two liquids to reach the end product. Your goal is to maximize the quality of the product in the end, which is measured by the difference between the Napierian logarithm of the product of the litres used of each component and the cube root of the summation of the first component litres as well as double the second component litres. It is required that the squared difference between the litres employed of each component and the mean litres utilised of both components is below ten. This is necessary for the obtained set to be stable. Furthermore, the mixture tank available to you can only fit 80 l. Please formulate a mathematical optimization model for this problem.  """

In [ ]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [ ]:
print(response1.choices[0].message.content)

Parameters:
Let c1 and c2 be the volumes (in litres) of the two components. 

Decision Variables:
x1 = Volume (in litres) of the first component used.
x2 = Volume (in litres) of the second component used.

Objective Function:
Our goal is to maximize the quality of the product. Thus, the objective function is:
max P = ln(x1 * x2) - cube_root(x1 + (2 * x2))

Subject to Constraints:
1) The squared difference between the volumes of each component used and their mean volume used must be less than ten:
(x1 - (x1+x2)/2) ^ 2 ≤ 10 
(x2 - (x1+x2)/2) ^ 2 ≤ 10 

2) The total volume of both components must not exceed the capacity of the tank:
x1 + x2 ≤ 80

3) Quantity used GTE 0:
x1 ≥ 0 
x2 ≥ 0 

This is a non-linear optimization problem and might need specific techniques or solvers for its solution.


### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [ ]:
Markdown(response2.choices[0].message.content)

Here is the Python Pyomo code for given optimization problem using the provided data:

```python
from pyomo.environ import *
from math import log, pow

#Creating a Concrete Model instance
model = ConcreteModel()

#Defining the decision variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)

#Mean value
mean_value = (model.x1 + model.x2) / 2

#Defining the objective function
model.profit = Objective(expr = log(model.x1 * model.x2) - pow((model.x1 + 2*model.x2), 1/3), sense=maximize)

#Adding the constraints 
model.Constraint1 = Constraint(expr = (model.x1 - mean_value)**2 <= 10)
model.Constraint2 = Constraint(expr = (model.x2 - mean_value)**2 <= 10)
model.Constraint3 = Constraint(expr = model.x1 + model.x2 <= 80)

#Applying solver
solver = SolverFactory('ipopt')
result = solver.solve(model, tee=True)

#Printing the optimal solutions
print("\n The Optimal Solution is:")
print('x1 = ', model.x1())
print('x2 = ', model.x2())
print('Profit = ', model.profit())
```

NOTE: 
Please replace the solver 'ipopt' according to the solver available in the system. Running this program may need to install the pyomo and ipopt.

Also, please note that this is a complex nonlinear optimization problem. The mathematical functions like `log()`, and `pow()` could potentially result in math domain error depending upon the search direction of the solver (`x1` and `x2` might become 0). Therefore, one could adjust the initial values of `x1` and `x2` or reformulate the problem for better numerical behavior of the solver.

### **Run the code Generated by GPT4**

In [ ]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 13.7 MB/s eta 0:00:00


In [ ]:

_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [ ]:
from pyomo.environ import *
from math import log, pow

# Creating a Concrete Model instance
model = ConcreteModel()

# Defining the decision variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)

# Mean value
mean_value = (model.x1 + model.x2) / 2

# Defining the objective function
model.profit = Objective(expr = log(model.x1 * model.x2) - pow((model.x1 + 2*model.x2), 1/3), sense=maximize)

# Adding the constraints
model.Constraint1 = Constraint(expr = (model.x1 - mean_value)**2 <= 10)
model.Constraint2 = Constraint(expr = (model.x2 - mean_value)**2 <= 10)
model.Constraint3 = Constraint(expr = model.x1 + model.x2 <= 80)

# Applying solver
solver = SolverFactory('ipopt')
result = solver.solve(model, tee=True)

# Printing the optimal solutions
print("\n The Optimal Solution is:")
print('x1 = ', model.x1())
print('x2 = ', model.x2())
print('Profit = ', model.profit())

TypeError: Implicit conversion of Pyomo numeric value (x1*x2) to float is disabled.
This error is often the result of using Pyomo components as arguments to
one of the Python built-in math module functions when defining
expressions. Avoid this error by using Pyomo-provided math functions or
explicitly resolving the numeric value using the Pyomo value() function.

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [ ]:
from pyomo.environ import *
from pyomo.core.expr.current import log

# Creating a Concrete Model instance
model = ConcreteModel()

# Defining the decision variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)

# Mean value
mean_value = (model.x1 + model.x2) / 2

# Defining the objective function
model.profit = Objective(expr = log(model.x1 * model.x2) - (model.x1 + 2*model.x2)**(1/3), sense=maximize)

# Adding the constraints
model.Constraint1 = Constraint(expr = (model.x1 - mean_value)**2 <= 10)
model.Constraint2 = Constraint(expr = (model.x2 - mean_value)**2 <= 10)
model.Constraint3 = Constraint(expr = model.x1 + model.x2 <= 80)

# Applying solver
solver = SolverFactory('knitro')
result = solver.solve(model, tee=True)

# Printing the optimal solutions
print("\n The Optimal Solution is:")
print('x1 = ', model.x1())
print('x2 = ', model.x2())
print('Profit = ', model.profit())

Artelys Knitro 14.0.0: 
          Commercial License
         Artelys Knitro 14.0.0


--- ERROR evaluating objective gradient.
ERROR: User routine for grad_callback returned -502.
       Could not evaluate first derivatives at the current point.
         No presolve will be applied.
concurrent_evals:        0
datacheck:               0
hessian_no_f:            1
presolve:                0
Knitro shifted start point further inside bounds (2 variables).

Problem Characteristics
-----------------------
Objective goal:  Maximize
Objective type:  general
Number of variables:                                  2
    bounded below only:                               2
    bounded above only:                               0
    bounded below and above:                          0
    fixed:                                            0
    free:                                             0
Number of constraints:                                3
    linear equalities:                              